In [1]:
import pandas as pd
df_train = pd.read_csv('clean_train.csv')
y = df_train['Survived']
df_train = df_train.drop(columns=['Survived'])
df_test = pd.read_csv('clean_test.csv')
df_train.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,TicketDigits,NanAge,NanFare,NanEmbarked,...,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T
0,1,3,22.0,1,0,7.2500,21171,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,1,38.0,1,0,71.2833,17599,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,3,26.0,0,0,7.9250,3101282,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,35.0,1,0,53.1000,113803,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,3,35.0,0,0,8.0500,373450,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
def print_moderate_correlations(df, col_to_correlate):
    cols = df[df.columns].corr().columns
    if df[col_to_correlate].dtype.name == 'category':
        df[col_to_correlate] = df[col_to_correlate].cat.codes
    corrs = df[df.columns].corr()[col_to_correlate]
    for col, corr in zip(cols, corrs):
        if abs(corr) >= 0.1 and col != col_to_correlate:
            print(col, ': ', corr)

In [3]:
df_train['Survived'] = y
print_moderate_correlations(df_train, 'Survived')
df_train = df_train.drop(columns=['Survived'])

Pclass :  -0.33848103596101586
Fare :  0.2573065223849618
TicketDigits :  -0.115156746957067
Infant :  0.12296550820907481
Toddler :  0.14899620582377618
Child :  0.11763628271102686
Adult :  -0.12766250958731493
Alone :  -0.2033670856998897
Sex_female :  0.5433513806577526
Sex_male :  -0.5433513806577526
Embarked_C :  0.16824043121823343
Embarked_S :  -0.14968272327068607
Title_Mr :  -0.5491991849030061
Title_Mrs :  0.3390402513843243
Title_Ms :  0.3299478186475593
FirstName_Anna :  0.12798529722217367
FirstName_Elizabeth :  0.1205973489336645
FirstName_Margaret :  0.10432228934184416
CabinLetter_B :  0.17509503365047693
CabinLetter_C :  0.11022636799098437
CabinLetter_D :  0.15071564423048353
CabinLetter_E :  0.14532144323642776


In [4]:
def remove_weak_correlations(df1: pd.DataFrame, df2, y, weak_threshold: float = 0.1) -> pd.DataFrame:
    """
    Removes weak correlations
    :param df: pandas DataFrame to remove columns from.
    :param col_to_correlate: String column name to check correlation with
    :param weak_threshold: float number that counts as an absolute weak threshold
    :return: pandas DataFrame without the columns weakly correlated to target
    """
    col_to_correlate = 'PREDICTOR_TO_CHECK_WEAK_CORRS'
    df1[col_to_correlate] = y
    cols = df1[df1.columns].corr().columns
    corrs = df1[df1.columns].corr()[col_to_correlate]
    strongly_correlated = list()
    for col, corr in zip(cols, corrs):
        if abs(corr) >= weak_threshold and col != col_to_correlate:
            strongly_correlated.append(col)
    df1 = df1.drop(columns=[col_to_correlate])
    for col in df1.columns:
        if col not in strongly_correlated:
            df1.drop(columns=[col], inplace=True)
            df2.drop(columns=[col], inplace=True)
    return df1, df2

In [5]:
def get_rid_feature(df, df2, feat):
    for column in df_train:
        if column.startswith(feat):
            df_train.drop(columns=[column], inplace=True)
            df_test.drop(columns=[column], inplace=True)
    return df_train, df_test

In [6]:
df_train, df_test = remove_weak_correlations(df_train, df_test, y)

In [7]:
df_train.head()

,Pclass,Fare,TicketDigits,Infant,Toddler,Child,Adult,Alone,Sex_female,Sex_male,...,Title_Mr,Title_Mrs,Title_Ms,FirstName_Anna,FirstName_Elizabeth,FirstName_Margaret,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E
0,3,7.2500,21171,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
1,1,71.2833,17599,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,3,7.9250,3101282,0,0,0,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
3,1,53.1000,113803,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4,3,8.0500,373450,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0


In [8]:
def print_moderate_correlations(df, col_to_correlate):
    cols = df[df.columns].corr().columns
    if df[col_to_correlate].dtype.name == 'category':
        df[col_to_correlate] = df[col_to_correlate].cat.codes
    corrs = df[df.columns].corr()[col_to_correlate]
    for col, corr in zip(cols, corrs):
        if abs(corr) > 0.1 and col != col_to_correlate:
            print(col, ': ', corr)

In [9]:
# df_train, df_test = get_rid_feature(df_train, df_test, 'TicketDigits')
# df_train, df_test = get_rid_feature(df_train, df_test, 'FirstName')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Sex_male')
# df_train, df_test = get_rid_feature(df_train, df_test, 'CabinLetter')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Title')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Embarked')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Infant')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Toddler')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Adult')

In [10]:
df_train.head()

,Pclass,Fare,TicketDigits,Infant,Toddler,Child,Adult,Alone,Sex_female,Sex_male,...,Title_Mr,Title_Mrs,Title_Ms,FirstName_Anna,FirstName_Elizabeth,FirstName_Margaret,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E
0,3,7.2500,21171,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
1,1,71.2833,17599,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,3,7.9250,3101282,0,0,0,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
3,1,53.1000,113803,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4,3,8.0500,373450,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.model_selection import train_test_split

x = df_train.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
import numpy as np
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)

In [14]:
from sklearn.model_selection import cross_val_score

def evaluate_classifier(clf, x, y):
    scores = cross_val_score(gb_clf, x, y, cv=5)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) 

In [15]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(criterion='friedman_mse', init=None,
             learning_rate=0.05, max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=12000,
             presort='auto', subsample=1.0, verbose=0,
             warm_start=False)

evaluate_classifier(gb_clf, X, Y)

Accuracy: 0.82 (+/- 0.04)


In [16]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

gb2_clf = GradientBoostingClassifier(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   random_state = 5)

evaluate_classifier(gb2_clf, X, Y)

Accuracy: 0.81 (+/- 0.04)


In [17]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(penalty='l2', tol=0.0001, solver='lbfgs',
                           C=1, fit_intercept=True, intercept_scaling=1.0, 
                           class_weight=None, random_state=None)

evaluate_classifier(lr_clf, X, Y)

Accuracy: 0.81 (+/- 0.03)


In [18]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(max_iter=1000, hidden_layer_sizes=(100,10), learning_rate='adaptive')

evaluate_classifier(mlp_clf, x_train, y_train)

Accuracy: 0.81 (+/- 0.03)


In [19]:
df_sub = pd.read_csv('gender_submission.csv')
df_sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [20]:
sub_model = gb_clf

In [21]:
gb_clf.fit(X, Y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=15, min_samples_split=10,
              min_weight_fraction_leaf=0.0, n_estimators=12000,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [22]:
df_sub['Survived'] = gb_clf.predict(df_test.values)
df_sub.to_csv('submission.csv', index=False)